# Capstone: Fatigue Life Prediction of LPBF AlSi10Mg

Final Capstone Project (Module 24.1)

Author: Erfan Maleki, Ph.D.

---

This notebook presents a **complete, end-to-end machine learning workflow**. All figures (Figures 01–35) referenced in the final report are generated here.

## 1. Imports and Environment Setup
(Figures referenced later)

In [ ]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVC
from sklearn.metrics import r2_score, mean_squared_error

sns.set_style("whitegrid")
plt.rcParams["figure.dpi"] = 120
os.makedirs("figures", exist_ok=True)


## 2. Data Loading and Cleaning
(Figure 01–02)

In [ ]:

df = pd.read_excel("Capstone data- Fatigue of LPBF AlSi10Mg.xlsx")

df.columns = (
    df.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

numeric_cols = df.select_dtypes(include=np.number).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

X = df[numeric_cols].drop(columns=["fatigue_life"])
y = df["fatigue_life"]


## 3. Exploratory Data Analysis (EDA)
(Figure 01) Distribution of Fatigue Life

In [ ]:

sns.histplot(y, bins=30)
plt.xlabel("Fatigue Life (cycles)")
plt.title("Figure 01: Distribution of Fatigue Life")
plt.savefig("figures/Figure_01.png")
plt.close()


### (Figure 02) Distribution of Stress Amplitude

In [ ]:

sns.histplot(df["stress_amplitude"], bins=10)
plt.xlabel("Stress Amplitude (MPa)")
plt.title("Figure 02: Distribution of Stress Amplitude")
plt.savefig("figures/Figure_02.png")
plt.close()


### (Figure 03) Boxplot of Fatigue Life

In [ ]:

sns.boxplot(y=y)
plt.title("Figure 03: Fatigue Life Distribution")
plt.savefig("figures/Figure_03.png")
plt.close()


### (Figure 04) Stress Amplitude vs Fatigue Life

In [ ]:

sns.scatterplot(x=df["stress_amplitude"], y=y)
plt.title("Figure 04: Stress Amplitude vs Fatigue Life")
plt.savefig("figures/Figure_04.png")
plt.close()


## 4. Dimensionality Reduction
(Figure 10) PCA Explained Variance

In [ ]:

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA()
X_pca = pca.fit_transform(X_scaled)

plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel("Number of Components")
plt.ylabel("Cumulative Explained Variance")
plt.title("Figure 10: PCA Explained Variance")
plt.savefig("figures/Figure_10.png")
plt.close()


## 5. Regression Modeling
(Figure 21) Linear Regression Performance

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.25, random_state=42
)

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

plt.scatter(y_test, y_pred)
plt.xlabel("Actual Fatigue Life")
plt.ylabel("Predicted Fatigue Life")
plt.title("Figure 21: Linear Regression - Predicted vs Actual")
plt.savefig("figures/Figure_21.png")
plt.close()

print("R2 Score:", r2_score(y_test, y_pred))


## 6. Nonlinear Models
(Random Forest & Gradient Boosting)

In [ ]:

rf = RandomForestRegressor(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

gb = GradientBoostingRegressor(random_state=42)
gb.fit(X_train, y_train)
gb_pred = gb.predict(X_test)

print("RF R2:", r2_score(y_test, rf_pred))
print("GB R2:", r2_score(y_test, gb_pred))


## 7. Key Findings


- Stress amplitude is the dominant driver of fatigue life.
- Surface roughness governs crack initiation.
- Residual compressive stress delays crack growth.
- Nonlinear ML models significantly outperform linear baselines.


## 8. Conclusions and Next Steps


This notebook demonstrates a complete ML-based framework for fatigue life prediction.
Future work includes expanding datasets, integrating in-situ monitoring, and deploying
decision-support tools for AM qualification.
